## Environment Setup

In [ ]:
!cd LMOps_MiniLLM/minillm && bash install.sh

Cloning into 'LMOps_MiniLLM'...
remote: Enumerating objects: 13340, done.
remote: Counting objects: 100% (756/756), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 13340 (delta 455), reused 378 (delta 361), pack-reused 12584 (from 5)
Receiving objects: 100% (13340/13340), 2.06 GiB | 34.30 MiB/s, done.
Resolving deltas: 100% (7152/7152), done.
  Cloning https://github.com/t1101675/transformers (to revision minillm) to /tmp/pip-req-build-84f9729q
  Running command git clone --filter=blob:none --quiet https://github.com/t1101675/transformers /tmp/pip-req-build-84f9729q
  Running command git checkout -b minillm --track origin/minillm
  Switched to a new branch 'minillm'
  branch 'minillm' set up to track 'origin/minillm'.
  Resolved https://github.com/t1101675/transformers to commit be0435edd0e45f491fbb66fe9fa630d458a2ace6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━

In [ ]:
# !huggingface-cli login --token <token>

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `model_upload` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `model_upload`


# Download Teacher and Student Model
Modify model names to train your own models

In [ ]:
!huggingface-cli download meta-llama/Llama-3.2-3B-Instruct --repo-type model --local-dir LMOps_MiniLLM/minillm/checkpoints/llama-3B # teacher model
!huggingface-cli download AlphaAnas70/pruned_llama-3.2-3b-Instruct-20_per --repo-type model --local-dir LMOps_MiniLLM/minillm/checkpoints/llama-prune # student model

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 16 files:   0%|                                 | 0/16 [00:00<?, ?it/s]Downloading 'generation_config.json' to 'LMOps_MiniLLM/minillm/checkpoints/llama-3B/.cache/huggingface/download/3EVKVggOldJcKSsGjSdoUCN1AyQ=.75ae08310d6d23df373ee2644b497192b3cce6d8.incomplete'

config.json: 100%|█████████████████████████████| 878/878 [00:00<00:00, 8.94MB/s]
Download complete. Moving file to LMOps_MiniLLM/minillm/checkpoints/llama-3B/config.json

generation_config.json: 100%|███████████████████| 189/189 [00:00<00:00, 400kB/s]

USE_POLICY.md:   0%|                                | 0.00/6.02k [00:00<?, ?B/s]Download complete. Moving file to LMOps_MiniLLM/minillm/checkpoints/llama-3B/generation_config.json
USE_POLICY.md: 100%|███████████████████████| 6.02k/6.02k [00:00<00:00, 22.7MB/s]
Download complete. Moving file to LMOps_MiniLLM/minillm/checkpoints/llama-3B/USE_POLICY.md

README.md: 100%|█████████████████████

### Process dataset

In [5]:
!huggingface-cli download MiniLLM/dolly --repo-type dataset --local-dir LMOps_MiniLLM/minillm/data/dolly/
!cd LMOps_MiniLLM/minillm && bash scripts/llama/tools/process_data_dolly.sh . # Process Dolly Train / Validation Data

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 6 files:   0%|                                   | 0/6 [00:00<?, ?it/s]Downloading 'valid.jsonl' to 'LMOps_MiniLLM/minillm/data/dolly/.cache/huggingface/download/n3KB4SjBGaOKaFnSW7Ur5AMEAYY=.f3d75af7b273550bc0ff340faffe17c8c31733d3.incomplete'

valid.txt: 0.00B [00:00, ?B/s]

.gitattributes: 2.46kB [00:00, 12.2MB/s]A
Download complete. Moving file to LMOps_MiniLLM/minillm/data/dolly/.gitattributes
Fetching 6 files:  17%|████▌                      | 1/6 [00:00<00:01,  4.85it/s]

valid.txt: 167kB [00:00, 17.4MB/s]A
Download complete. Moving file to LMOps_MiniLLM/minillm/data/dolly/valid.txt

raw.txt: 0.00B [00:00, ?B/s]Downloading 'README.md' to 'LMOps_MiniLLM/minillm/data/dolly/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.1005769f61bee0bb423c64d289f08c1003e153d7.incomplete'
valid.jsonl: 506kB [00:00, 9.62MB/s]
Download complete. Moving file to LMOps_MiniLLM/minillm/data/dolly/valid.jso

### Setup Training

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# for terminal
# export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

## MiniLLM

In [ ]:
!cd LMOps_MiniLLM/minillm && bash scripts/llama/minillm/train_prune_3B_lora.sh .
# or run in terminal

torchrun --nproc_per_node 1 --nnodes 1 --node_rank 0 --master_addr localhost --master_port 2012 ./train_minillm.py --base-path . --model-path ./checkpoints/llama-prune/ --teacher-model-path ./checkpoints/llama-3B/ --ckpt-name llama-prune --teacher-ckpt-name llama-3B --n-gpu 1 --n-nodes 1 --model-type llama --teacher-model-fp16 --gradient-checkpointing --prompt-data-dir ./processed_data/dolly/prompt/llama/ --lm-data-dir ./processed_data/roberta/llama/512/20M/ --dev-num 1000 --num-workers 0 --epochs 10 --total-iters 5000 --kd-ratio 0.5 --batch-size 8 --lr 5e-6 --lr-min 5e-6 --gradient-accumulation-steps 2 --max-length 512 --max-prompt-length 256 --warmup-iters 100 --scheduler-name cosine_trm --save ./results/llama/train/minillm/ --seed 10 --seed-ppo 42 --seed-lm 7 --save-interval 500 --eval-interval 100 --log-interval 16 --mid-log-num 1 --teacher-peft-name lora-3B --teacher-peft-path ./results/llama/train/lora-3B/ --type minillm --ppo-epochs 4 --num-rollouts 256 --chunk-size 8 --length-n

## Save model
Provide a path below to the final trained model. Path will be similar to:
```LMOps_MiniLLM/minillm/results/llama/train/minillm/bs8-lr5e-06-G2-N1-NN1-lm1-len512/pe4_rs0.5_nr32_ln_sr_tm0.2/300/```

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Step 1: Load model
model_id = ""# your model path

# Step 2: Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
repo_name = "llama_3b_pruned_mini" # Choose a name for your repository
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)